In [21]:
!python -m pip install --upgrade pip



Defaulting to user installation because normal site-packages is not writeable


In [ ]:
!pip install -q fastapi uvicorn streamlit chromadb langchain \
sentence-transformers transformers torch \
python-jose passlib[bcrypt] pandas
!pip install tf-keras
!pip install -U langchain langchain-community langchain-core
from langchain_community.document_loaders import TextLoader, CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document


In [ ]:
import os
folders = [
    "backend",
    "backend/db",
    "backend/auth",
    "backend/rag",
    "frontend",
    "data"]
for f in folders:
    os.makedirs(f, exist_ok=True)
print("Project structure created")

In [ ]:
from langchain.document_loaders import TextLoader, CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
import glob

ROLE_MAP = {
    "finance": ["Finance"],
    "marketing": ["Marketing"],
    "hr": ["HR"],
    "engineering": ["Engineering"],
    "general": ["Employees"],
    "all": ["C-Level"]
}
documents = []
for file in glob.glob("data/**/*", recursive=True):
    if file.endswith(".md") or file.endswith(".txt"):
        loader = TextLoader(file)
    elif file.endswith(".csv"):
        loader = CSVLoader(file)
    else:
        continue
    docs = loader.load()
    role = "general"

    for k in ROLE_MAP:
        if k in file.lower():
            role = k
    for d in docs:
        d.metadata["role"] = ROLE_MAP[role]
        d.metadata["source"] = file
        documents.append(d)
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(documents)
print(f"Total chunks: {len(chunks)}")


In [ ]:
from langchain.document_loaders import TextLoader, CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
import glob

ROLE_MAP = {
    "finance": ["Finance"],
    "marketing": ["Marketing"],
    "hr": ["HR"],
    "engineering": ["Engineering"],
    "general": ["Employees"],
    "all": ["C-Level"]
}

documents = []

for file in glob.glob("data/**/*", recursive=True):
    if file.endswith(".md") or file.endswith(".txt"):
        loader = TextLoader(file)
    elif file.endswith(".csv"):
        loader = CSVLoader(file)
    else:
        continue

    docs = loader.load()
    role = "general"

    for k in ROLE_MAP:
        if k in file.lower():
            role = k

    for d in docs:
        d.metadata["role"] = ROLE_MAP[role]
        d.metadata["source"] = file
        documents.append(d)

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(documents)

print(f"Total chunks: {len(chunks)}")


In [ ]:
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma

embedding_model = SentenceTransformerEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vectordb = Chroma.from_documents(
    chunks,
    embedding_model,
    persist_directory="backend/db/chroma"
)

vectordb.persist()
print("Vector DB ready")


In [ ]:
%%writefile backend/auth/security.py
from datetime import datetime, timedelta
from jose import jwt
from passlib.context import CryptContext
SECRET_KEY = "Sindhusai_3"
ALGORITHM = "123456789"
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")
def hash_password(pw):
    return pwd_context.hash(pw)
def verify_password(pw, hashed):
    return pwd_context.verify(pw, hashed)

def create_token(data):
    payload = data.copy()
    payload["exp"] = datetime.utcnow() + timedelta(hours=4)
    return jwt.encode(payload, SECRET_KEY, algorithm=ALGORITHM)
